# **Proceso ETL de dato COVID-19**
---
Dentro de la empresa es necesario la obtencion de los datos del informe COVID-19 entregado por la autoridad sanitaria, para su posterior analisis a traves de graficos estadisticos.

Los datos obtenidos provienen de repositorios pertenecientes al Ministerio de Ciencia y a Ivan Gonzales, ambos repositorios alojados en GitHub.

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

Los datos que se obtienen pertenecen a los datos de casos activos y contagios diarios a nivel nacional, regiona y comunal, dentro de un periodo de tiempo establecido.

## Ingreso de parametros
---

Para comenzar con el proceso, es necesario ingresar el numero de dias para determinar el periodo de tiempo de estudio.
El periodo de tiempo se establece desde el dia de ayer hasta "x" dias atras, **por ejemplo, la fecha actual es 13-01-2022 y se establecen 30 dias de estudio, los datos obtenidos perteneceran al periodo entre 12-01-2022 y 13-12-2021, ambos inclusive.**

In [2]:
numero_dias = int(input("Ingrese el numero de dias: "))

In [3]:
periodo = dt.timedelta(days=numero_dias)
fecha_final = dt.date.today() - dt.timedelta(days=1)
fecha_inicial = fecha_final - periodo
nombre_hoja = "Hoja1"

## Funciones necesarias
---

*Funcion que setea un valor de la tupla en nulo si el valor es igual a la palabra ingresada*

In [4]:
def set_tupla(tupla,lista,columna):
  if not tupla[columna] in lista:
    tupla[columna] = np.nan
  return tupla

*Funcion que retorna un date a partir de una fecha en string*

In [5]:
def string_date(fecha_str):
  fecha = fecha_str.split("-")
  if (len(fecha[0]) > 2):
    fecha_str = fecha_str.split("-")
    return dt.date(int(fecha_str[0]),int(fecha_str[1]),int(fecha_str[2]))
  return dt.date(int(fecha[2]),int(fecha[1]),int(fecha[0]))

*Funcion para setear los sw a None si una tupla no corresponde al periodo de fechas indicados, respecto a la columna asignada*

In [6]:
def set_sw(tupla,fecha1,fecha2,columna):
  if tupla[columna] < fecha1 or tupla[columna] > fecha2:
    tupla["sw"] = None
  return tupla

*Corrobora si una fecha es par o no*

In [7]:
def is_par(fecha):
    return fecha.day % 2 == 0

*Reemplaza las vocales con tilde*

In [8]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

*Reemplaza los tildes de las palabras de una lista*

In [9]:
def reemplazar_tildes(lista):
    columnas = list(lista)
    for i in range(len(columnas)):
        columnas[i] = normalize(columnas[i])
    return columnas

## Archivo Contagios Diarios Regional
---

En esta seccion se comienza a extraer los datos del archivo 'CasosNuevosCumulativos_T.csv' para su posterior tratamiento.

In [10]:
df_casos_nuevos_acumulativos = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto13/CasosNuevosCumulativo_T.csv')

Se transforma las fechas de string a date, luego se crea una columna switch con valor 0 para eliminar posteriormente los datos no necesarios

In [11]:
df_casos_nuevos_acumulativos["Region"] = df_casos_nuevos_acumulativos["Region"].apply(string_date)
df_casos_nuevos_acumulativos = df_casos_nuevos_acumulativos.assign(sw=0)

Se cambian los valores 0 a nulo los datos que estan fuera del rango de fechas, todo esto en la columna 'sw'

In [12]:
df_casos_nuevos_acumulativos = df_casos_nuevos_acumulativos.apply(set_sw,args=[fecha_inicial,fecha_final,"Region"],axis=1)

Se eliminan los valores nulos respecto a la columna 'sw', luego esta es eliminada

In [13]:
df_casos_nuevos_acumulativos = df_casos_nuevos_acumulativos.dropna(subset=["sw"])
df_casos_nuevos_acumulativos = df_casos_nuevos_acumulativos.drop(columns=["sw"])

Se eliminan los tildes de los nombres de las columnas

In [14]:
df_casos_nuevos_acumulativos.columns = reemplazar_tildes(df_casos_nuevos_acumulativos.columns)

Se extrae el data frame a formato excel

In [15]:
df_casos_nuevos_acumulativos.to_excel("ContagiosDiariosRegional.xlsx",sheet_name=nombre_hoja,index=False)

## Archivo Recuperados Activos Acumulado
---

Se extraen los datos del archivo 'TotalesNacionales_T.csv'

In [16]:
df_totales_nacionales = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv')

Se reemplazan los valores nulos a 0, ya que, corresponden a datos que no fueron registrados al comienzo de la pandemia por el Minsal.

In [17]:
df_totales_nacionales = df_totales_nacionales.replace(np.nan,0)

Se guarda el nombre de las columnas necesarias del data frame

In [18]:
columnas = ["Fecha","Casos nuevos con sintomas","Casos totales","Casos recuperados","Fallecidos","Casos activos","Casos nuevos sin sintomas","Casos nuevos totales","Casos activos por FD",
            "Casos activos por FIS","Casos recuperados por FIS","Casos recuperados por FD","Casos confirmados recuperados","Casos activos confirmados",
            "Casos probables acumulados","Casos activos probables","Casos nuevos sin notificar","Casos confirmados por antigeno","Casos con sospecha de reinfeccion",
           "Casos nuevos confirmados por antigeno"]

El data frame es ordenado al formato anterior y se igualan las columna por pertenecer al mismo dato

In [19]:
df_totales_nacionales1 = df_totales_nacionales[columnas]
df_totales_nacionales1["Casos recuperados"] = df_totales_nacionales1["Casos confirmados recuperados"]

Se transforma la fecha de string a date

In [20]:
df_totales_nacionales1["Fecha"] = df_totales_nacionales1["Fecha"].apply(string_date)

Se crea la columna switch con valor 0, se setea el switch segun si la tupla pertenece al rango de fechas. Luego se elimina los nulos y la columna 'sw'

In [21]:
df_totales_nacionales1 = df_totales_nacionales1.assign(sw=0)
df_totales_nacionales1 = df_totales_nacionales1.apply(set_sw,args=[fecha_inicial,fecha_final,"Fecha"],axis=1)
df_totales_nacionales1 = df_totales_nacionales1.dropna(subset=["sw"])
df_totales_nacionales1 = df_totales_nacionales1.drop(columns=["sw"])

Se renombran las columnas para seguir el formato ya definido

In [22]:
df_totales_nacionales1.columns = ['Fecha', 'Casos nuevos con sintomas', 'Casos totales',
       'Casos recuperados', 'Fallecidos', 'Casos activos',
       'Casos nuevos sin sintomas', 'Casos nuevos',
       'Casos activos por FD', 'Casos activos por FIS',
       'Casos recuperados por FIS', 'Casos recuperados por FD',
       'Casos confirmados recuperados', 'Casos activos confirmados',
       'Casos probables acumulados', 'Casos activos probables',
       'Casos nuevos sin notificar', 'Casos confirmados por antigeno',
       'Casos con sospecha de reinfeccion',
       'Casos nuevos confirmados por antigeno']

Se exportan los datos del data frame a un excel

In [23]:
df_totales_nacionales1.to_excel("RecuperadosActivosAcumulado.xlsx",sheet_name=nombre_hoja,index=False)

## Archivo Comunas acumulado
---

Extraccion de datos del archivo 'historial_comuna.csv'

In [24]:
df_historial_comunas = pd.read_csv('https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/historial_comunas.csv')

Se formatea la fecha del data frame

In [25]:
df_historial_comunas["Fecha"] = df_historial_comunas["Fecha"].apply(string_date)

In [26]:
fechas = list(df_historial_comunas["Fecha"])
indices = list(df_historial_comunas.index)
current = fecha_inicial
current_RM = fecha_inicial
for i in range(len(fechas)):
    if current < fechas[i] and df_historial_comunas.loc[indices[i],"Region"] != "RM":
        current = fechas[i]
    if current_RM < fechas[i] and df_historial_comunas.loc[indices[i],"Region"] == "RM":
        current_RM = fechas[i]
print(current,current_RM)

2022-02-21 2022-02-18


Se crea 2 data frame para diferenciar las comunas de la region metropolitana y las comunas de las demas regiones

In [27]:
df_HC_RM = pd.DataFrame(columns=df_historial_comunas.columns)
df_HC_otras_regiones = pd.DataFrame(columns=df_historial_comunas.columns)

Se agregan las tuplas a su respectivo data frame

In [28]:
columnas = df_historial_comunas.columns
for i in df_historial_comunas.index:
    if df_historial_comunas.loc[i,"Region"] == "RM" and df_historial_comunas.loc[i,"Fecha"] == current_RM:
        df_HC_RM = df_HC_RM.append({columnas[0] : df_historial_comunas.loc[i,"Fecha"],columnas[1] : df_historial_comunas.loc[i,"CUT"],columnas[2] : df_historial_comunas.loc[i,"Region"],columnas[3] : df_historial_comunas.loc[i,"Comuna"],columnas[4] : df_historial_comunas.loc[i,"Confirmados Acumulados"],columnas[5] : df_historial_comunas.loc[i,"Nuevos Confirmados"]},ignore_index=True)
    elif df_historial_comunas.loc[i,"Region"] != "RM" and df_historial_comunas.loc[i,"Fecha"] == current:
        df_HC_otras_regiones = df_HC_otras_regiones.append({columnas[0] : df_historial_comunas.loc[i,"Fecha"],columnas[1] : df_historial_comunas.loc[i,"CUT"],columnas[2] : df_historial_comunas.loc[i,"Region"],columnas[3] : df_historial_comunas.loc[i,"Comuna"],columnas[4] : df_historial_comunas.loc[i,"Confirmados Acumulados"],columnas[5] : df_historial_comunas.loc[i,"Nuevos Confirmados"]},ignore_index=True)

Se eliminan los tildes dentro de todas las palabras posibles que puedan llevar tilde y se crea un DataFrame vacio

In [29]:
df_historial_comunas = pd.DataFrame(columns=df_HC_otras_regiones.columns)
df_HC_otras_regiones["Comuna"] = reemplazar_tildes(df_HC_otras_regiones["Comuna"])
df_HC_RM["Comuna"] = reemplazar_tildes(df_HC_RM["Comuna"])

Se dejan los datos en un solo DataFrame

In [30]:
df_historial_comunas = pd.concat([df_historial_comunas,df_HC_otras_regiones],ignore_index=True)
df_historial_comunas = pd.concat([df_historial_comunas,df_HC_RM],ignore_index=True)

Se crea un archivo excel en donde se almacenan los datos en 3 hojas distintas

In [31]:
df_historial_comunas.to_excel("ComunasAcumulado.xlsx",sheet_name=nombre_hoja,index=False)

## Archivo activos comunales
---

Se extraen los datos del archivo 'CasosPorComuna.csv', se reemplazan los datos nulos por 0 y se duplica el data frame con terminacion periodo

In [32]:
df_casos_activos_comuna = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv')
df_casos_activos_comuna = df_casos_activos_comuna.replace(np.nan,0)
df_casos_activos_comuna_periodo = df_casos_activos_comuna.copy()

Se extrae la ultima fecha registrada

In [33]:
columnas = list(df_casos_activos_comuna.columns[5:].copy())
fecha_activos = string_date(columnas[len(columnas)-1])

Se formatea las fechas de las columnas

In [34]:
columnas = list(df_casos_activos_comuna.columns)
for i in range(len(columnas[5:])):
    fecha = string_date(columnas[5 + i])
    if fecha < fecha_inicial or fecha > fecha_final:
        df_casos_activos_comuna_periodo[columnas[5 + i]] = np.nan
    columnas[5 + i] = fecha.strftime('%B_%d').capitalize()

Cambia el nombre de las columnas

In [35]:
df_casos_activos_comuna_periodo.columns = columnas
df_casos_activos_comuna.columns = columnas

Elimina los nulos

In [36]:
df_casos_activos_comuna_periodo = df_casos_activos_comuna_periodo.dropna(axis=1)

Se reemplaza las tildes de la columna region

In [37]:
df_casos_activos_comuna["Region"] = reemplazar_tildes(df_casos_activos_comuna["Region"])
df_casos_activos_comuna_periodo["Region"] = reemplazar_tildes(df_casos_activos_comuna_periodo["Region"])

Se crea otro dataFrame con los datos de los casos activos de la ultima fecha registrada

In [38]:
fechas = list(df_casos_activos_comuna_periodo.columns[5:].copy())
columnas = list(df_casos_activos_comuna_periodo.columns[:4].copy())
columnas.append(fechas[len(fechas)-1])
df_activos_presente = df_casos_activos_comuna_periodo[columnas]
df_activos_presente.columns = ['Region', 'Codigo region', 'Comuna', 'Codigo comuna', 'Activos']
df_activos_presente = df_activos_presente.assign(Fecha=fecha_activos)

Se crea un archivo excel con los datos obtenidos

In [39]:
df_casos_activos_comuna.to_excel("ActivosComunales.xlsx",sheet_name="Historial",index=False)
with pd.ExcelWriter("ActivosComunales.xlsx", mode="a", engine="openpyxl") as writer:
      df_casos_activos_comuna_periodo.to_excel(writer, sheet_name="grafico",index=False)
      df_activos_presente.to_excel(writer,sheet_name=nombre_hoja,index=False)

## Archivo evolucion COVID
---

Se extraen los datos del archivo 'covid19_chile.csv'

In [14]:
df_evolucion_covid = pd.read_csv('https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/covid19_chile.csv')

Se ordena el data frame, se formatean las fechas a formato date y se eliminan las tuplas que no perteneces al rango de fechas

In [15]:
df_evolucion_covid = df_evolucion_covid[['Fecha', 'Region', 'Nuevo Confirmado', 'Nuevo Muerte',
       'Nuevo Recuperado', 'Acum Confirmado', 'Acum Muerte', 'Acum Recuperado']]
df_evolucion_covid = df_evolucion_covid.assign(sw=0)
df_evolucion_covid["Fecha"] = df_evolucion_covid["Fecha"].apply(string_date)
df_evolucion_covid = df_evolucion_covid.apply(set_sw,args=[fecha_inicial,fecha_final,"Fecha"],axis=1)
df_evolucion_covid = df_evolucion_covid.dropna(subset=["sw"])
df_evolucion_covid = df_evolucion_covid.drop(columns=["sw"])

Se actualizan los datos de la columna 'Nuevos confirmados' desde el archivo del producto 4, iterando por cada archivo desde la fecha inicial hasta la fecha final del periodo establecido

In [16]:
fecha_current = fecha_inicial
regiones = list(pd.unique(df_evolucion_covid["Region"]))
df_evolucion_covid = df_evolucion_covid.reset_index(drop=True)
index = 0
for i in range(periodo.days + 1):
    df_current = pd.read_csv(f'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/{fecha_current}-CasosConfirmados-totalRegional.csv')
    df_current = df_current.apply(set_tupla,args=[regiones,"Region"],axis=1)
    df_current = df_current.dropna(subset=["Region"])
    for a in df_current.index:
        df_evolucion_covid.loc[index,"Nuevo Confirmado"] = df_current.loc[a,"Casos nuevos totales"]
        index += 1
    fecha_current = fecha_current + dt.timedelta(days=1)

Se reemplazan los tildes

In [17]:
df_evolucion_covid.columns = reemplazar_tildes(df_evolucion_covid.columns)
df_evolucion_covid["Region"] = reemplazar_tildes(df_evolucion_covid["Region"])

In [18]:
regiones = list(pd.unique(df_evolucion_covid["Region"]))
regiones.pop()
df_evolucion_covid = df_evolucion_covid.apply(set_tupla,args=[regiones,"Region"],axis=1)
df_evolucion_covid = df_evolucion_covid.dropna(subset=["Region"])

Se crea el archivo excel con los datos

In [46]:
df_evolucion_covid.to_excel("EvolucionCovid.xlsx",sheet_name=nombre_hoja,index=False)

## Archivo activos totales regional
---

Se crea un data frame vacio para agregar datos en él

In [47]:
df_casos_activos_region = pd.DataFrame(columns=['Region','Fecha','ACTIVOS'])

Se agregan los datos desde el producto 4 al data frame vacio

In [48]:
fecha_current = fecha_inicial
for i in range(periodo.days + 1):
    df_current = pd.read_csv(f'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/{fecha_current}-CasosConfirmados-totalRegional.csv')
    df_current = df_current.assign(Fecha=fecha_current)
    df_current = df_current[['Region','Fecha','Casos activos confirmados']]
    for a in df_current.index:
        df_casos_activos_region = df_casos_activos_region.append({'Region':df_current.loc[a,'Region'],'Fecha':df_current.loc[a,"Fecha"],'ACTIVOS':df_current.loc[a,'Casos activos confirmados']},ignore_index=True)
    fecha_current = fecha_current + dt.timedelta(days=1)

Se crean dos data frame, para agregar en uno las fechas pares y en otro las fechas impares

In [49]:
df_CAR_par = pd.DataFrame(columns=df_casos_activos_region.columns)
df_CAR_impar = pd.DataFrame(columns=df_casos_activos_region.columns)

Se separan los datos en su respectivo data frame

In [50]:
columnas = df_casos_activos_region.columns
for i in df_casos_activos_region.index:
    if is_par(df_casos_activos_region.loc[i,"Fecha"]):
        df_CAR_par = df_CAR_par.append({columnas[0] : df_casos_activos_region.loc[i,"Region"],columnas[1]:df_casos_activos_region.loc[i,"Fecha"],columnas[2]:df_casos_activos_region.loc[i,"ACTIVOS"]},ignore_index=True)
    else:
        df_CAR_impar = df_CAR_impar.append({columnas[0] : df_casos_activos_region.loc[i,"Region"],columnas[1]:df_casos_activos_region.loc[i,"Fecha"],columnas[2]:df_casos_activos_region.loc[i,"ACTIVOS"]},ignore_index=True)

Se eliminan las tuplas que no correspondes al rango de fechas

In [51]:
df_CAR_impar = df_CAR_impar.assign(sw=0)
df_CAR_impar = df_CAR_impar.apply(set_sw,args=[fecha_inicial,fecha_final,"Fecha"],axis=1)
df_CAR_par = df_CAR_par.assign(sw=0)
df_CAR_par = df_CAR_par.apply(set_sw,args=[fecha_inicial,fecha_final,"Fecha"],axis=1)
df_CAR_impar = df_CAR_impar.dropna(subset=["sw"])
df_CAR_par = df_CAR_par.dropna(subset=["sw"])
df_CAR_impar = df_CAR_impar.drop(columns=["sw"])
df_CAR_par = df_CAR_par.drop(columns=["sw"])

Se reemplazan los tildes de las Regiones

In [52]:
df_casos_activos_region["Region"] = reemplazar_tildes(df_casos_activos_region["Region"])
df_CAR_impar["Region"] = reemplazar_tildes(df_CAR_impar["Region"])
df_CAR_par["Region"] = reemplazar_tildes(df_CAR_par["Region"])


Se crea un excel en donde se almacenan los datos de los data frame

In [53]:
df_casos_activos_region.to_excel("ActivosTotalRegional.xlsx",sheet_name="Master",index=False)
with pd.ExcelWriter("ActivosTotalRegional.xlsx", mode="a", engine="openpyxl") as writer:
      df_CAR_par.to_excel(writer, sheet_name="MasterPar",index=False)
      df_CAR_impar.to_excel(writer, sheet_name="MasterImpar",index=False)

## Archivo reporte diario minsal
---

Se extraen dos archivos de datos, el archivo de la fecha actual del producto 4 y el archivo 'CasosActivosPorComuna.csv'

In [10]:
df_reporte_diario = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv')
df_producto4 = pd.read_csv(f'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/{fecha_final}-CasosConfirmados-totalRegional.csv')

Se ordena el dataframe y se elimina las tuplas que no corresponden

In [11]:
df_reporte_diario = df_reporte_diario[["Codigo region","Region","Poblacion","Comuna"]]
df_reporte_diario = df_reporte_diario.apply(set_tupla,args=[["Total"],"Comuna"],axis=1)
df_reporte_diario = df_reporte_diario.dropna(subset=["Comuna"])
df_reporte_diario = df_reporte_diario.drop(columns=["Comuna"])



se eliminan las tuplas no corresponden

In [19]:
df_producto4["Region"] = reemplazar_tildes(df_producto4["Region"])
df_producto4 = df_producto4.apply(set_tupla,args=[regiones,"Region"],axis=1)
df_producto4 = df_producto4.dropna(subset=["Region"])

Se aplica reestablece el indice de las tuplas

In [20]:
df_reporte_diario = df_reporte_diario.reset_index(drop=True)

Se extraen los datos del producto4 al dataframe reporte diario

In [21]:
df_reporte_diario["Casos totales"] = df_producto4["Casos totales acumulados"]
df_reporte_diario["Casos nuevos"] = df_producto4["Casos nuevos totales"]
df_reporte_diario["Casos nuevos con sintomas"] = df_producto4["Casos nuevos con sintomas"]
df_reporte_diario["Casos nuevos sin sintomas"] = df_producto4["Casos nuevos sin sintomas*"]
df_reporte_diario["Activos"] = df_producto4["Casos activos confirmados"]
df_reporte_diario["Fallecidos"] = df_producto4["Fallecidos totales"]
df_reporte_diario["Recuperados"] = df_producto4["Casos confirmados recuperados"]


Se renombran y ordenan las columnas

In [22]:
df_reporte_diario.columns = ['COD_REGION', 'Region', 'POBLACION', 'Casos totales', 'Casos nuevos','Casos nuevos con sintomas', 'Casos nuevos sin sintomas', 'Activos','Fallecidos', 'Recuperados']
df_reporte_diario = df_reporte_diario[['COD_REGION','Region','Casos totales','Casos nuevos','Casos nuevos con sintomas','Casos nuevos sin sintomas','Activos','Fallecidos','Recuperados','POBLACION']]

*Se añade la tupla de la suma de cada columna*

In [41]:
sumas = list(df_reporte_diario.sum(axis=0,numeric_only=True))
sumas[0] = "-"
sumas.insert(1,"Total")
columnas = df_reporte_diario.columns
df_reporte_diario = df_reporte_diario.append(dict(zip(columnas,sumas)),ignore_index=True)

Se exportan los datos en un archivo excel

In [42]:
df_reporte_diario.to_excel("ReporteDiarioMinsal.xlsx",sheet_name=nombre_hoja,index=False)